<center>
    
## 3.2. Pandas advanced. Составляем отчетность.

Как и всегда начинаем с импорта библиотек.

In [10]:
import numpy as np
import pandas as pd

### Немного парсинга.

Также импортируем библиотки для парсинга.<br>
* [**requests**](http://docs.python-requests.org/en/master/) позволяет посылать запросы и получать данные, например html страницы и их содержимое.
* [**BeautifulSoup**](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) универсальная библиотека для парсинга, получение данных из html, поиск по тегам.
* [**re**](https://docs.python.org/3/library/re.html)  - модуль python который позволяет обрабатывать данные, например строки и чистить их, искать паттерны и прочее. <br>
С **re** все ограничивается вашей фантазией.

In [18]:
import requests
from bs4 import BeautifulSoup 
import re

Итак, для целей задачи я нашел соответствие кодов названиям валют в интернете.<br>
Искал более менее удобные данные, чтобыы их можно было быстро собрать и сложить в табличку, <br>
чтобы потом распространить на нашу главную таблицу с транзакциями.

In [19]:
url = 'https://en.wikipedia.org/wiki/ISO_4217'

Далее мы передаем наш урл в в библиотку requests и просим получить все что с ней ассоциированно.

In [23]:
page = requests.get(url)
print(page, type(page))

<Response [200]> <class 'requests.models.Response'>


Далее создаем "суп" и текста объекта, тоесть новый объект с которым уже могут работать методы библиотеки `BeautifulSoup`.

In [27]:
soup = BeautifulSoup(page.text, 'html.parser')
print(soup, type(soup))

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>ISO 4217 - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"c80b48fa-e196-457d-890e-64af6a0850e1","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"ISO_4217","wgTitle":"ISO 4217","wgCurRevisionId":1062341578,"wgRevisionId":1062341578,"wgArticleId":15403,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 Spanish-language sources (es)","Articles with short description","Short description is different from Wikidata","Use dmy dates from November 2014","Articles containi

В коде страницы, который можно простомотреть в инструментах разработчика я заметил, что табличка с кодами, лежит на википедии в тэге `table` и конкретно принадлежит классу `wikitable sortable`.

In [30]:
table = soup.find('table',{'class':'wikitable sortable collapsible mw-collapsible mw-made-collapsible jquery-tablesorter'})

In [29]:
print(table)

None


Так как все значения содержатся в тегах вида `td>...</td>` и данные представлены у нас по строчно, то я предположил что можно взять первый элемет в нашем массиве в определенном классе, который мы ходим найти, например коды валют и через определенное число строк мы наткнемся на следующий код валюты, который идет у нас в таблице на википедии. <br>
Пулем подбора и шаманства с бубном я смог подобрать шаг, чтобы составить 3 листа: коды валют, текстовые коды валют и текстовое поле, где лежит полная текстовая расшифровка (например *русский рубль*).<br>

Также я использовал метод `.findAll`, который очевидно ищет все объекты(строки), которые содержатся в тегах, которые можно передать этому методу в качестве аргумента.

In [9]:
currency_codes = table.findAll('td')[::5]
number_codes = table.findAll('td')[1::5]
titles = table.findAll('td')[3::5]

AttributeError: 'NoneType' object has no attribute 'findAll'

Код для создания таблички из данных, которые мы спарсили с подробной документацией.

In [36]:
""" 
Создаем 3 пустых списка для добавления уже очищенных данных, чтобы можно было складывать очищенные значения
без лишней путаницы. На этом часто не акцентирую внимание, но старайтесь делать все как можно проще. Например. 
при создании переменной подумайте как ее назвать понятно, логично и не длинно. Если похожих переменных будет много, 
то начните с общего для них слова, чтобы они были единообразны, как например ниже.
"""
clean_currency_codes = []
clean_number_codes = []
clean_titles = []

"""
Итак, в этом цикле мы склеиваем по-элементно наши необработанные списки при помощи метода zip и для каждого 
из списка проводим некие манипуляции по очищению. 
Так как наша искомая строка лежит в тегах вида td>...</td>, то мы просто разрем ее по символам, получая
список с объектами из изначальной строки.
Далее мы индексируемся/выбираем нужный нам объект, где содержится искомая строка и, повтораяя эту процедуру еще раз, 
мы добираемся до самого значения, убирая тэги. Чистое значение добавляем в пустые списки.
Для заголовков у нас существует 2 кейса:
1) Когда есть ссылка и тогда мы достаем название через регулярное выражение и тэг title;
2) Ссылки нет и все это лежит как обычная строка в тэгах <td>...</td> и тут мы повторяем известные нам манипуляции,
описанные выше.
"""

for currency_code,number_code,title in zip(currency_codes,number_codes,titles):
    clean_currency_code = str(currency_code).split('>')[1].split('<')[0]
    clean_currency_codes.append(clean_currency_code)

    clean_number_code = str(number_code).split('>')[1].split('<')[0]
    clean_number_codes.append(clean_number_code)

    if 'title' in str(title):
        refined_string = re.findall('title="(.+?)"',str(title))
        clean_titles.append(refined_string[-1])
    else:
        refined_string = str(title).split('>')[1].split('<')[0]
        clean_titles.append(refined_string)

В финале, мы собираем наши очищенные списки в таблицу и чувствуем себя большими молодцами.<br>
Выглядит прямо как на [странице википедии](https://en.wikipedia.org/wiki/ISO_4217).

In [43]:
wiki_table = pd.DataFrame({'num_code':clean_number_codes,'currency_code':clean_currency_codes,'title':clean_titles},
             columns=['num_code','currency_code','title'])
wiki_table.head()

,num_code,currency_code,title
0,784,AED,United Arab Emirates dirham
1,971,AFN,Afghan afghani
2,008,ALL,Albanian lek
3,051,AMD,Armenian dram
4,532,ANG,Netherlands Antillean guilder


Записываем в папку с данными, например в формате `csv`.

In [45]:
wiki_table.to_csv('../data/mean_check/currency_codes_wiki.csv', index=False)